### Here I have used AWS Textract API for geteing text from images.
### I have tried aws rekognition API as well.
### It is better than aws rekognition API Because it support only 50 word sequences which results in loss of text in some cases.


In [1]:
import boto3
import csv
import re
import pandas as pd
from calendar import month_abbr
from datetime import datetime
import os

In [2]:
# Establishing connection with Rekognition API
# Credentials.csv is the file which includes access key and id 
with open('credentials.csv','r') as input:
    next(input)
    reader=csv.reader(input)
    for line in reader:
        access_key_id=line[2]
        secret_access_key=line[3]

In [3]:
client=boto3.client('textract',aws_access_key_id=access_key_id,aws_secret_access_key=secret_access_key)


In [4]:
path='/home/sak/DL/material/text_ocr/RC/RC'  #Path to folder where images are stored
final_path=os.listdir('/home/sak/DL/material/text_ocr/RC/RC')  #Final_path stores list of all images in above path
final_data=[]  # A list to store extracted information

In [5]:
for file in final_path:
    photo = os.path.join(path,file)
    print("Under Process:-",file)
    with open(photo,'rb') as source_image:
        source_bytes=source_image.read()
    response=client.detect_document_text(Document={'Bytes':source_bytes})
    temp=[]
    for item in response['Blocks']:
        if item["BlockType"] == "LINE":
            temp.append(item['Text'])
    #print(temp)    #  It stores extracted text
    # A dictionary to store information of extracted fields.
    dicc={"Registration_no":"Not Found","Engine_no":"Not Found","Chasis_no":"Not Found","Register_date":"Not Found","Name":"Not Found","Manufacturing Date":"Not Found"}
    
    m=13
    for i in range(0,len(temp)):
        
        field=temp[i]
        
        #  Regex for Engine No.
        if re.findall(r'\b([A-Z0-9]{4}[0-9]{7})\b',temp[i]):
            dicc['Engine_no']=re.findall(r'\b([A-Z0-9]{4}[0-9]{7})\b',temp[i])[0]
        elif re.findall(r'\b([0-9]{7,9})\b',temp[i]):
            dicc['Engine_no']=re.findall(r'\b([0-9]{7,9})\b',temp[i])[0]
            
            
        # Regex for Chasis Number
        if re.findall(r'\b([A-Z0-9]{14,})\b',temp[i]):
            dicc['Chasis_no']=re.findall(r'\b([A-Z0-9]{14,})\b',temp[i])[0]
            continue

            
        # Regex for Registration Number
        if re.findall(r'\b([A-Z]{2}[0-9A-Z]{4}[0-9]{4})\b',temp[i]):
            dicc["Registration_no"]=re.findall(r'\b([A-Z]{2}[0-9A-Z]{4}[0-9]{4})\b',temp[i])[0]
            continue
        if re.findall(r'\b([A-Z]{2}[0-9A-Z]{3}[\s][0-9]{4}|[A-Z]{1}[0-9A-Z]{4,5}[\s][0-9]{4})\b',temp[i]):
            dicc["Registration_no"]=re.findall(r'\b([A-Z]{2}[0-9A-Z]{3}[\s][0-9]{4}|[A-Z]{1}[0-9A-Z]{4,5}[\s][0-9]{4})\b',temp[i])[0]
            continue
        if re.findall(r'\b([A-Z]{2}[0-9A-Z]{2}[-][A-Z]{1}[-][0-9]{4})\b',temp[i]):
            dicc["Registration_no"]=re.findall(r'\b([A-Z]{2}[0-9A-Z]{2}[-][A-Z]{1}[-][0-9]{4})\b',temp[i])[0]
            continue
            
            
        ## Name of person ##
        if temp[i]=='NAME' or temp[i]=='Name' or temp[i]=='Name & Address':
            if temp[i+1]=='I' and temp[i+2]:
                dicc['Name']=temp[i+2]
            else:
                dicc['Name']=temp[i+1]
            continue
        if (re.search('NAME', field)) is not None and dicc['Name']=='Not Found': 
            field = field[re.search('NAME', field).end():]
            name=re.search("[a-zA-Z].*[a-zA-Z]",field)
            if name is not None:
                name = field[name.start():name.end()]
                dicc['Name']=name
            continue
        # TRY:-2 #
        if re.search('Name', field) is not None and dicc['Name']=='Not Found' : 
            field = field[re.search('Name', field).end():]
            name=re.search("[a-zA-Z].*[a-zA-Z]",field)
            if name is not None:
                name = field[name.start():name.end()]
                dicc['Name']=name
            continue
            
        
        ##  Second Attempt for Engine No.
       
        if re.findall(r'\b([A-Z0-9]{5}[0-9]{6,7})\b',temp[i]):
            dicc['Engine_no']=re.findall(r'\b([A-Z0-9]{5}[0-9]{6,7})\b',temp[i])[0]
        if re.findall(r'\b([A-Z0-9]{6}[0-9]{6})\b',temp[i]):
            dicc['Engine_no']=re.findall(r'\b([A-Z0-9]{6}[0-9]{6})\b',temp[i])[0]
        if re.findall(r'[A-Z][0-9]\w{3,4}\s?\d{5,6}',temp[i]):
            dicc['Engine_no']=re.findall(r'[A-Z][0-9]\w{3,4}\s?\d{5,6}',temp[i])[0]  
        
        
            
        ## Registeration Date
        if re.findall(r"\d{1,2}[/-]\w{2,}[/-]\d{4}",temp[i]):
            s=re.findall(r"\d{1,2}[/-]\w{2,}[/-]\d{4}",temp[i])
            for j in s:
                if '-' in j:
                    day, month, year = j.split("-")
                    for k, v in enumerate(month_abbr):
                        if v == month:
                            month = k
                            break
                    m=int(month)
                else:
                    day, month, year = j.split("/")
                    for k, v in enumerate(month_abbr):
                        if v == month:
                            month = k
                            break
                    m=int(month)
                if int(year)<2020:
                    dicc['Register_date']=j
           
       # Regex logic Manufacturing Date 
      # Also checking that it shiul not be greater than registration date
        if re.findall(r"[^\/]\d{1,2}[/]\d{4}",temp[i]):
            s=re.findall(r"[^\/]\d{1,2}[/]\d{4}",temp[i])
            for j in s:
                if '-' in j:
                    month, year = j.split("-")
                else:
                    month, year = j.split("/")
                    
                if month.isnumeric() and int(month) <int(m):
                    dicc['Manufacturing Date']=j
                    #print("1")
                        
        if temp[i]=='MFG.DT' or temp[i]=='MFG.DT.':
            if re.match(r"[^\/]\d{1,2}[/]\d{4}",temp[i+1]):
                dicc['Manufacturing Date']=temp[i+1]
                #print("2")
        if temp[i].find('Mfg.') or temp.find('MFG'):
            eng=re.search("[^\/]\d{1,2}[/]\d{4}",temp[i])
            if eng is not None:
                eng = temp[i][eng.start():eng.end()]
                for j in s:
                    if '-' in j:
                        month, year = j.split("-")
                    else:
                        month, year = j.split("/")

                    if int(year)<2020:
                        dicc['Manufacturing Date']=j

                #print("3")
    final_data.append(dicc)
    #print(dicc)
            
        


Under Process:- txt_mudit_b11_961.jpg
Under Process:- txt_mudit_b11_11599.jpg
Under Process:- txt_mudit_b8_1_808.jpg
Under Process:- txt_mudit_b11_746.jpg
Under Process:- txt_mudit_b11_1328.jpg
Under Process:- txt_mudit_b11_1347.jpg
Under Process:- txt_mudit_b8_1_854.jpg
Under Process:- txt_mudit_b8_2_213.jpg
Under Process:- txt_mudit_b11_1334.jpg
Under Process:- txt_mudit_b8_1_742.jpg
Under Process:- txt_mudit_b8_1_571.jpg
Under Process:- txt_mudit_b8_1_554.jpg
Under Process:- txt_mudit_b8_1_574.jpg
Under Process:- txt_mudit_b11_1355.jpg
Under Process:- txt_mudit_b8_1_807.jpg
Under Process:- txt_mudit_b11_1529.jpg
Under Process:- txt_mudit_b11_1539.jpg
Under Process:- txt_mudit_b8_1_546.jpg
Under Process:- txt_mudit_b11_1339.jpg
Under Process:- txt_mudit_b11_1534.jpg
Under Process:- txt_mudit_b11_11597.jpg
Under Process:- txt_mudit_b11_1343.jpg
Under Process:- txt_mudit_b11_1332.jpg
Under Process:- txt_mudit_b8_1_806.jpg
Under Process:- txt_mudit_b11_480.jpg
Under Process:- txt_mudit_

In [6]:
final_data[0]   #Sample extracted information from first image

{'Registration_no': 'DL5CJ 6088',
 'Engine_no': 'D13A2235055',
 'Chasis_no': 'MA3FHEB1S00520199',
 'Register_date': '06/10/2013',
 'Name': 'RAJNISH JAIN',
 'Manufacturing Date': '09/2013'}

### Code to write dictionary values to excel file

In [9]:
from xlsxwriter import Workbook
ordered_list=["Registration_no","Engine_no","Chasis_no","Register_date","Name",'Manufacturing Date'] #list object calls by index but dict object calls items randomly
wb=Workbook("Final_Output.xlsx")
ws=wb.add_worksheet("New Sheet") #or leave it blank, default name is "Sheet 1"
first_row=0
for header in ordered_list:
    col=ordered_list.index(header) # we are keeping order.
    ws.write(first_row,col,header) # we have written first row which is the header of worksheet also.
row=1
for j in final_data: 
    #print(j)
    for _key,_value in j.items():
        col=ordered_list.index(_key)
        ws.write(row,col,_value)
    row+=1 #enter the next row
wb.close()

In [11]:
pd.read_excel("Final_Output.xlsx")
# Preview of Excel file

,Registration_no,Engine_no,Chasis_no,Register_date,Name,Manufacturing Date
0,DL5CJ 6088,D13A2235055,MA3FHEB1S00520199,06/10/2013,RAJNISH JAIN,09/2013
1,DL9CAC6215,D13A0338461,MA3FHEB1S0035858,24/12/2012,- SRISHTI NAYAR,12/2012
2,Not Found,Not Found,Not Found,19/05/2010,Brand of Vehicle,05/2010
3,Not Found,371772454,Not Found,08/02/2013,Not Found,01/7213
4,HRO6P 5988,F8DN3321864,MA3EYD81S00765439,07-Feb-2007,SUBE SINGH,9/2006
5,DL2CAU7997,D13A2554860,MA3FLEB1S00309631,02/05/2015,MANJEET SINGH,03/2015
6,HR10-M-5657,D13A1120011,MA3FKEB2S00121766,29/08/2008,ANUBHAV JAIN,08/2008
7,DL8CP 1150,F10DN3294185,MA3EMD81S00167856,20/12/2007,ADITYA GUPTA,12/2007
8,HR06AH1768,K14BN 712469,MA3ELMG1800384268,Not Found,MR HARISH KUMAR,4/2016
9,DL3CAD3216,G13BBN 190141,Not Found,31/01/2005,MAHABIR SINGH,01/2005
